### EXTRACCION DE LOS DATOS BEAUTIFUL SOUP

In [2]:
import pandas as pd ##Pandas nunca falta 
from lxml import html ##lxml para leer paginas en html
from string import ascii_uppercase as alfabeto
import pickle

In [3]:
## Aca se trae los documentos html en este caso se quiere extraer las tablas vacias de el mundial 2022
todas_tablas=pd.read_html("https://web.archive.org/web/20221115040351/https://en.wikipedia.org/wiki/2022_FIFA_World_Cup", encoding="utf-8")

In [4]:
todas_tablas[12]

,Pos,"Team.mw-parser-output .navbar{display:inline;font-size:88%;font-weight:normal}.mw-parser-output .navbar-collapse{float:left;text-align:left}.mw-parser-output .navbar-boxtext{word-spacing:0}.mw-parser-output .navbar ul{display:inline-block;white-space:nowrap;line-height:inherit}.mw-parser-output .navbar-brackets::before{margin-right:-0.125em;content:""[ ""}.mw-parser-output .navbar-brackets::after{margin-left:-0.125em;content:"" ]""}.mw-parser-output .navbar li{word-spacing:-0.125em}.mw-parser-output .navbar a>span,.mw-parser-output .navbar a>abbr{text-decoration:inherit}.mw-parser-output .navbar-mini abbr{font-variant:small-caps;border-bottom:none;text-decoration:none;cursor:inherit}.mw-parser-output .navbar-ct-full{font-size:114%;margin:0 7em}.mw-parser-output .navbar-ct-mini{font-size:114%;margin:0 4em}vte",Pld,W,D,L,GF,GA,GD,Pts,Qualification
0,1,Qatar (H),0,0,0,0,0,0,0,0,Advance to knockout stage
1,2,Ecuador,0,0,0,0,0,0,0,0,Advance to knockout stage
2,3,Senegal,0,0,0,0,0,0,0,0,NaN
3,4,Netherlands,0,0,0,0,0,0,0,0,NaN


In [5]:
#A-->H
#Extraccion de tablas de la pagina web, cambia cada 7 indices 
todas_tablas[61]

,Pos,Teamvte,Pld,W,D,L,GF,GA,GD,Pts,Qualification
0,1,Portugal,0,0,0,0,0,0,0,0,Advance to knockout stage
1,2,Ghana,0,0,0,0,0,0,0,0,Advance to knockout stage
2,3,Uruguay,0,0,0,0,0,0,0,0,NaN
3,4,South Korea,0,0,0,0,0,0,0,0,NaN


In [6]:
alfabeto

'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [7]:
## uso de la funcion ZIP para aplicar la letra del grupo a cada dataframe 
for letra, i in zip (alfabeto,range(11,67,7)):
    print(letra,i)

A 11
B 18
C 25
D 32
E 39
F 46
G 53
H 60


In [8]:
todas_tablas=pd.read_html("https://web.archive.org/web/20221115040351/https://en.wikipedia.org/wiki/2022_FIFA_World_Cup", encoding="utf-8")
dict_tablas={} ##diccionario vacio que va a permitir almacenar los dataframes
for letra , i in zip(alfabeto, range(12,68,7)):  ## Se hace un for para extraer todas las columnas 
    df= todas_tablas[i]
    df.rename(columns={df.columns[1]:"Team"}, inplace=True) ## aca se quiere cambiar las columnas Teamvte a Team 
    df.pop("Qualification") ## se elimina la columna Qualification 
    dict_tablas[f'Grupo{letra}']=df ##aca se añaden los grupos a el diccionario 

In [9]:
dict_tablas.keys() ##funcion keys para devolver todos los elementos que son grupos

dict_keys(['GrupoA', 'GrupoB', 'GrupoC', 'GrupoD', 'GrupoE', 'GrupoF', 'GrupoG', 'GrupoH'])

In [10]:
dict_tablas["GrupoA"]

,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts
0,1,Qatar (H),0,0,0,0,0,0,0,0
1,2,Ecuador,0,0,0,0,0,0,0,0
2,3,Senegal,0,0,0,0,0,0,0,0
3,4,Netherlands,0,0,0,0,0,0,0,0


In [11]:
## exportar Diccionario con libreria pickle, para poder utilizarlo en donde quiera 
with open("dict_table","wb") as output:
    pickle.dump(dict_tablas,output)

### TODAVIA NO SE QUE PONER ACA 

In [12]:
from bs4 import BeautifulSoup as BS
import requests

In [17]:
web="https://en.wikipedia.org/wiki/2014_FIFA_World_Cup"
respons =requests.get(web)
content= respons.text
soup=BS(content,"lxml")

matches =soup.find_all("div",class_="footballbox")  ##identificacion de patro para extraer info dentro de una pagina web 

## Creacion de listas vacias para guardar los resultados de la extraccion 
home=[]
score=[]
away=[]
## Este for lo que hace es extraer el nombre de los equipos y guardarlos en las listas vacias creadas anteriormente 
for match in matches:
    home.append(match.find('th',class_="fhome").get_text()) ##Extraer nombre del equipo local
    score.append(match.find('th',class_="fscore").get_text()) ##Extraer puntaje
    away.append(match.find('th',class_="faway").get_text())  ##Extraer nombre equipo visitante    

In [18]:
## Creacion de diccionario para guardar las listas creadas en el punto anterior
dict_football = {"home":home,"score":score,"away":away}
## se crea un data frame con el nombre del diccionario
df_football=pd.DataFrame(dict_football)
## Se añade una columna al dataframe como año, el cual representa el año de donde se saco la info 
df_football["year"]="2014"

In [19]:
df_football

,home,score,away,year
0,Brazil,3–1,Croatia,2014
1,Mexico,1–0,Cameroon,2014
2,Brazil,0–0,Mexico,2014
3,Cameroon,0–4,Croatia,2014
4,Cameroon,1–4,Brazil,2014
...,...,...,...,...
59,Netherlands,0–0 (a.e.t.),Costa Rica,2014
60,Brazil,1–7,Germany,2014
61,Netherlands,0–0 (a.e.t.),Argentina,2014
62,Brazil,0–3,Netherlands,2014


In [20]:
## listo el mundial 2014, pero ahora se quiere sacar toda la info de todos los mundiales, para ello 
## Se crea una funcion: 
def get_matches(year):
    web= f"https://en.wikipedia.org/wiki/{year}_FIFA_World_Cup"
    respons =requests.get(web)
    content= respons.text
    soup=BS(content,"lxml")

    matches =soup.find_all("div",class_="footballbox")  ##identificacion de patro para extraer info dentro de una pagina web 

## Creacion de listas vacias para guardar los resultados de la extraccion 
    home=[]
    score=[]
    away=[]
## Este for lo que hace es extraer el nombre de los equipos y guardarlos en las listas vacias creadas anteriormente 
    for match in matches:
        home.append(match.find('th',class_="fhome").get_text()) ##Extraer nombre del equipo local
        score.append(match.find('th',class_="fscore").get_text()) ##Extraer puntaje
        away.append(match.find('th',class_="faway").get_text())  ##Extraer nombre equipo visitante   
    ## Creacion de diccionario para guardar las listas creadas en el punto anterior
    dict_football = {"home":home,"score":score,"away":away}
## se crea un data frame con el nombre del diccionario
    df_football=pd.DataFrame(dict_football)
## Se añade una columna al dataframe como año, el cual representa el año de donde se saco la info 
    df_football["year"]=year 
    return df_football


In [29]:
## Aca se puede ver que si funciona al momento de insertar un año cualquiera! :) 
print(get_matches(1950))

              home score            away  year
0          Brazil    4–0          Mexico  1950
1      Yugoslavia    3–0     Switzerland  1950
2          Brazil    2–2     Switzerland  1950
3      Yugoslavia    4–1          Mexico  1950
4          Brazil    2–0      Yugoslavia  1950
5     Switzerland    2–1          Mexico  1950
6         England    2–0           Chile  1950
7           Spain    3–1   United States  1950
8           Spain    2–0           Chile  1950
9   United States    1–0         England  1950
10          Spain    1–0         England  1950
11          Chile    5–2   United States  1950
12         Sweden    3–2           Italy  1950
13         Sweden    2–2        Paraguay  1950
14          Italy    2–0        Paraguay  1950
15        Uruguay    8–0         Bolivia  1950
16        Uruguay    2–2           Spain  1950
17         Brazil    7–1          Sweden  1950
18         Brazil    6–1           Spain  1950
19        Uruguay    3–2          Sweden  1950
20         Sw

In [30]:
## extraccion de todos los partidos de todos los años ! 
years=[1930,1934,1938,1950,1954,1958,1962,1966,1970,1974,1978,1982,1986,1990,1994,1998,2002,2006,2010,2014,2018]
fifa=[get_matches(year) for year in years]
df_fifa = pd.concat(fifa,ignore_index=True)## ayuda a concatenar los dataframes que se van creando 
    

In [32]:
## Se pasa el dataframe a un archivo csv 
df_fifa.to_csv("fifa_historical_data.csv",index=False)

In [40]:
## Extraer el fixture del 2022

web= f"https://web.archive.org/web/20221115040351/https://en.wikipedia.org/wiki/2022_FIFA_World_Cup"
respons =requests.get(web)
content= respons.text
soup=BS(content,"lxml")

matches =soup.find_all("div",class_="footballbox")  ##identificacion de patro para extraer info dentro de una pagina web 

## Creacion de listas vacias para guardar los resultados de la extraccion 
home=[]
score=[]
away=[]
## Este for lo que hace es extraer el nombre de los equipos y guardarlos en las listas vacias creadas anteriormente 
for match in matches:
    home.append(match.find('th',class_="fhome").get_text()) ##Extraer nombre del equipo local
    score.append(match.find('th',class_="fscore").get_text()) ##Extraer puntaje
    away.append(match.find('th',class_="faway").get_text())  ##Extraer nombre equipo visitante   
    ## Creacion de diccionario para guardar las listas creadas en el punto anterior
dict_fixture_2022 = {"home":home,"score":score,"away":away}
## se crea un data frame con el nombre del diccionario
df_fixture_2022=pd.DataFrame(dict_fixture_2022)
## Se añade una columna al dataframe como año, el cual representa el año de donde se saco la info 
df_fixture_2022["year"]=2022
   

In [43]:
df_fixture_2022.to_csv("Fifa_worldcup_fixture.csv",index=False)